In [0]:
import pandas as pd
import re
import string
import ast
import functools
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict, Counter
from googletrans import Translator
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import itertools
import numpy as np
import spacy
import os
import xml.etree.ElementTree as ET

In [0]:
pd.set_option('display.max_colwidth', -1)

In [0]:
tweet27_8 = pd.read_csv('D:\\Università\\Magistrale\\Secondo anno\\Social Media Analytics\\Progetto\\sentiment\\DatasetPreprocessato_noLemma.csv')

In [0]:
tweet27_8['token']=tweet27_8['token'].apply(lambda x: ast.literal_eval(x)) #trasforma lista
tweet27_8['tokenNoNeg']=tweet27_8['tokenNoNeg'].apply(lambda x: ast.literal_eval(x)) #trasforma lista

In [0]:
tweet27_8[['text','token','tokenNoNeg']]

,text,token,tokenNoNeg
0,"Comunque la si pensi, #Zalone riesce ancora una volta a riempire le sale di tutti i cinema italiani. In un settore da anni in grande difficoltà, lui riesce ad attirare spettatori di ogni età e classe sociale. È questa la grande forza di Checco: unire. #ToloTolo https://t.co/mUzLVlPxL0","[comunque, pensi, zalone, riesce, ancora, volta, riempire, sale, cinema, italiani, settore, anni, grande, difficoltà, riesce, attirare, spettatori, ogni, età, classe, sociale, grande, forza, checco, unire, tolotolo]","[comunque, la, si, pensi, ,, zalone, riesce, ancora, una, volta, a, riempire, le, sale, di, tutti, i, cinema, italiani, ., in, un, settore, da, anni, in, grande, difficoltà, ,, lui, riesce, ad, attirare, spettatori, di, ogni, età, e, classe, sociale, ., è, questa, la, grande, forza, di, checco, :, unire, ., tolotolo]"
1,"Non è ""diverso"" #ToloTolo. Sono i fan di #Zalone che finora non hanno capito un cazz*!","[non_diverso, non_tolotolo, fan, zalone, finora, non_capito, non_cazz]","[non, è, `, `, diverso, ', ', tolotolo, ., sono, i, fan, di, zalone, che, finora, non, hanno, capito, un, *, cazz, !]"
2,@matteosalvinimi @marcocf69 Sei in brodo di giugiule e tutta farina del tuo sacco fai solo la vittima na sei contento come #Zalone che parlavano male di #ToloTolo e guarda che successo .non fare la vittima!!💩,"[brodo, giugiule, tutta, farina, sacco, solo, vittima, na, contento, zalone, parlavano, male, tolotolo, guarda, successo, non_fare, non_vittima, 💩]","[sei, in, brodo, di, giugiule, e, tutta, farina, del, tuo, sacco, fai, solo, la, vittima, na, sei, contento, come, zalone, che, parlavano, male, di, tolotolo, e, guarda, che, successo, ., non, fare, la, vittima, !, !, 💩]"
3,"Ci siete rimasti male ""cari"" Sovranisti=Razzisti\ndei miei stivali😅\n\nLa rosicata dei sovranisti trollati da #ToloTolo di #CheccoZalone\n\nhttps://t.co/6GKEMRVQJv di @neXtquotidiano \n\n#facciamorete ❤️🇮🇹👊 https://t.co/3UJtrKExAQ","[rimasti, male, cari, sovranisti, razzisti, 😅, stivali, rosicata, sovranisti, trollati, tolotolo, checcozalone, facciamorete, ❤, ️, 👊]","[ci, siete, rimasti, male, `, `, cari, ', ', sovranisti, =, razzisti, dei, miei, 😅, stivali, la, rosicata, dei, sovranisti, trollati, da, tolotolo, di, checcozalone, di, facciamorete, ❤, ️, 👊]"
4,Salvini propone #CheccoZalone come senatore a vita dopo aver visto il trailer di #ToloTolo \nIl film si rivela una fantasmagorica presa per il culo del salvinismo \nZalone sparisce dai tweet di bandierini e sovranisti \nPiù esilarante di oggi le comiche,"[salvini, propone, checcozalone, senatore, vita, dopo, aver, visto, trailer, tolotolo, film, rivela, fantasmagorica, presa, culo, salvinismo, zalone, sparisce, twet, bandierini, sovranisti, esilarante, oggi, comiche]","[salvini, propone, checcozalone, come, senatore, a, vita, dopo, aver, visto, il, trailer, di, tolotolo, il, film, si, rivela, una, fantasmagorica, presa, per, il, culo, del, salvinismo, zalone, sparisce, dai, tweet, di, bandierini, e, sovranisti, più, esilarante, di, oggi, le, comiche]"
...,...,...,...
21565,"Il prezzo del biglietto di #ToloTolo vale anche solo per il cameo di Nichi Vendola (che capiranno in pochissimi, come il vero senso del film)","[prezzo, biglietto, tolotolo, vale, solo, cameo, nichi, vendola, capiranno, pochissimi, vero, senso, film]","[il, prezzo, del, biglietto, di, tolotolo, vale, anche, solo, per, il, cameo, di, nichi, vendola, (, che, capiranno, in, pochissimi, ,, come, il, vero, senso, del, film, )]"
21566,Qualcuno si offende se affermo che #ToloTolo fa cagare?,"[qualcuno, offende, affermo, tolotolo, fa, cagare]","[qualcuno, si, offende, se, affermo, che, tolotolo, fa, cagare, ?]"
21567,"Siccome sul film di Zalone manca sempre l’opinione degli afrodiscendenti e quindi dei diretti interessati per ciò che concerne la rappresentazione dei corpi neri nei media e cinema italiani, ecco alcuni passaggi del post di Andi Nganso, medico e attivista:\n#ToloTolo https://t.co/SNYVUAx0dJ","[siccome, 

In [0]:
os.environ['TREETAGGER_HOME']='C:\\TreeTagger'

In [0]:
import treetaggerwrapper as ttw
tagger = ttw.TreeTagger(TAGLANG='it')

In [0]:
def elimina_emoticon(token):
    if len(re.findall(r'[^\w\s,]', token))!=0:
        token=' '
    return token

def elimina_rumore(lemma):
    lemma_conv=str(lemma)
    if len(re.findall('^NotTag',lemma_conv))!=0:
        lemma=''
    return [lemma]

def crea_dizionario(lista):
    dizionario = defaultdict(list)
    lista_norumore=list(map(elimina_rumore, lista))
    lista_norumore=list(itertools.chain(*lista_norumore))
    lista_norumore=list(filter(None,lista_norumore))
    
    for parola, tipo, lemma in lista_norumore:
        if lemma!='replaced-url':
            
            if '|' in lemma:
                lemma=lemma.split('|')[1]
                
            dizionario[parola].append(lemma)
    return dizionario

def elimina_duplicati(dizionario):
    for key, value in dizionario.items():
        lemma_singolo = list(dict.fromkeys(value))
        lemma_singolo = str.split(lemma_singolo[0],"|")[0]
        dizionario[key] = lemma_singolo
    return dizionario
        
def lemmatization(token):
    tag = tagger.tag_text(token)
    lista_lemma = ttw.make_tags(tag)
    diz_lemma = crea_dizionario(lista_lemma)
    diz_lemma = elimina_duplicati(diz_lemma)
    return diz_lemma

In [0]:
inizio = datetime.now()
print("Inizio: "+inizio.strftime('%H:%M:%S'))

tweet27_8['dictLemma'] = tweet27_8.apply(lambda x: lemmatization(x['tokenNoNeg']),axis=1)

fine = datetime.now()
print("Fine: "+fine.strftime('%H:%M:%S'))
diff = fine - inizio
diff

Inizio: 15:21:49
Fine: 15:23:12


datetime.timedelta(0, 82, 901912)

In [0]:
tweet27_8[['text','token','tokenNoNeg','dictLemma']].head()

,text,token,tokenNoNeg,dictLemma
0,"Comunque la si pensi, #Zalone riesce ancora una volta a riempire le sale di tutti i cinema italiani. In un settore da anni in grande difficoltà, lui riesce ad attirare spettatori di ogni età e classe sociale. È questa la grande forza di Checco: unire. #ToloTolo https://t.co/mUzLVlPxL0","[comunque, pensi, zalone, riesce, ancora, volta, riempire, sale, cinema, italiani, settore, anni, grande, difficoltà, riesce, attirare, spettatori, ogni, età, classe, sociale, grande, forza, checco, unire, tolotolo]","[comunque, la, si, pensi, ,, zalone, riesce, ancora, una, volta, a, riempire, le, sale, di, tutti, i, cinema, italiani, ., in, un, settore, da, anni, in, grande, difficoltà, ,, lui, riesce, ad, attirare, spettatori, di, ogni, età, e, classe, sociale, ., è, questa, la, grande, forza, di, checco, :, unire, ., tolotolo]","{'comunque': 'comunque', 'la': 'la', 'si': 'si', 'pensi': 'pensare', ',': ',', 'zalone': 'zalone', 'riesce': 'riuscire', 'ancora': 'ancora', 'una': 'una', 'volta': 'volta', 'a': 'a', 'riempire': 'riempire', 'le': 'il', 'sale': 'sale', 'di': 'di', 'tutti': 'tutto', 'i': 'il', 'cinema': 'cinema', 'italiani': 'italiano', '.': '.', 'in': 'in', 'un': 'un', 'settore': 'settore', 'da': 'da', 'anni': 'anno', 'grande': 'grande', 'difficoltà': 'difficoltà', 'lui': 'lui', 'ad': 'ad', 'attirare': 'attirare', 'spettatori': 'spettatore', 'ogni': 'ogni', 'età': 'età', 'e': 'e', 'classe': 'classe', 'sociale': 'sociale', 'è': 'essere', 'questa': 'questo', 'forza': 'forza', 'checco': 'checco', ':': ':', 'unire': 'unire', 'tolotolo': 'tolotolo'}"
1,"Non è ""diverso"" #ToloTolo. Sono i fan di #Zalone che finora non hanno capito un cazz*!","[non_diverso, non_tolotolo, fan, zalone, finora, non_capito, non_cazz]","[non, è, `, `, diverso, ', ', tolotolo, ., sono, i, fan, di, zalone, che, finora, non, hanno, capito, un, *, cazz, !]","{'non': 'non', 'è': 'essere', '`': '`', 'diverso': 'diverso', ''': ''', 'tolotolo': 'tolotolo', '.': '.', 'sono': 'essere', 'i': 'il', 'fan': 'fan', 'di': 'di', 'zalone': 'zalone', 'che': 'che', 'finora': 'finora', 'hanno': 'avere', 'capito': 'capire', 'un': 'un', '*': '*', 'cazz': 'cazz', '!': '!'}"
2,@matteosalvinimi @marcocf69 Sei in brodo di giugiule e tutta farina del tuo sacco fai solo la vittima na sei contento come #Zalone che parlavano male di #ToloTolo e guarda che successo .non fare la vittima!!💩,"[brodo, giugiule, tutta, farina, sacco, solo, vittima, na, contento, zalone, parlavano, male, tolotolo, guarda, successo, non_fare, non_vittima, 💩]","[sei, in, brodo, di, giugiule, e, tutta, farina, del, tuo, sacco, fai, solo, la, vittima, na, sei, contento, come, zalone, che, parlavano, male, di, tolotolo, e, guarda, che, successo, ., non, fare, la, vittima, !, !, 💩]","{'sei': 'sei', 'in': 'in', 'brodo': 'brodo', 'di': 'di', 'giugiule': 'giugiule', 'e': 'e', 'tutta': 'tutto', 'farina': 'farina', 'del': 'del', 'tuo': 'tuo', 'sacco': 'sacco', 'fai': 'fare', 'solo': 'solo', 'la': 'il', 'vittima': 'vittima', 'na': 'na', 'contento': 'contento', 'come': 'come', 'zalone': 'zalone', 'che': 'che', 'parlavano': 'parlare', 'male': 'male', 'tolotolo': 'tolotolo', 'guarda': 'guardare', 'successo': 'successo', '.': '.', 'non': 'non', 'fare': 'fare', '!': '!', '💩': '💩'}"
3,"Ci siete rimasti male ""cari"" Sovranisti=Razzisti\ndei miei stivali😅\n\nLa rosicata dei sovranisti trollati da #ToloTolo di #CheccoZalone\n\nhttps://t.co/6GKEMRVQJv di @neXtquotidiano \n\n#facciamorete ❤️🇮🇹👊 https://t.co/3UJtrKExAQ","[rimasti, male, cari, sovranisti, razzisti, 😅, stivali, rosicata, sovranisti, trollati, tolotolo, checcozalone, facciamorete, ❤, ️, 👊]","[ci, siete, rimasti, male, `, `, cari, ', ', sovranisti, =, razzisti, dei, miei, 😅, stivali, la, rosicata, dei, sovranisti, trollati, da, tolotolo, di, checcozalone, di, facciamorete, ❤, ️, 👊]","{'ci': 'ci', 'siete': 'essere', 'rimasti': 'rimanere', 'male': 'male', '`': '`', 'cari': 'caro', ''': ''', 'sovranisti': 'sovranisti', '=': '=', 'razzisti': '

In [0]:
def ritornaLemma(dictLemma,token):
    neg=False
    stringa=token
    if 'non_non' in token:
        elimina='non_non'
        token=token.replace('non_','')
        
    token=token.replace('non_','')
    if stringa!=token:
        neg=True
    token_mod=dictLemma[token]
    token_mod=str(token_mod)
    if neg==True:
        token_mod = 'non_'+token_mod
        
    return [token_mod]

In [0]:
tweet27_8['tokenLemma']=tweet27_8.apply(lambda x: list(map(functools.partial(ritornaLemma, x['dictLemma']),x['token'])),axis=1)
tweet27_8['tokenLemma']=tweet27_8['tokenLemma'].apply(lambda x: list(itertools.chain(*x)))

In [0]:
tweet27_8['tokenLemma']=tweet27_8['tokenLemma'].apply(lambda x: [elemento for elemento in x if elemento != '[]'])

In [0]:
tweet27_8[['token','tokenLemma']].head()

,token,tokenLemma
0,"[comunque, pensi, zalone, riesce, ancora, volta, riempire, sale, cinema, italiani, settore, anni, grande, difficoltà, riesce, attirare, spettatori, ogni, età, classe, sociale, grande, forza, checco, unire, tolotolo]","[comunque, pensare, zalone, riuscire, ancora, volta, riempire, sale, cinema, italiano, settore, anno, grande, difficoltà, riuscire, attirare, spettatore, ogni, età, classe, sociale, grande, forza, checco, unire, tolotolo]"
1,"[non_diverso, non_tolotolo, fan, zalone, finora, non_capito, non_cazz]","[non_diverso, non_tolotolo, fan, zalone, finora, non_capire, non_cazz]"
2,"[brodo, giugiule, tutta, farina, sacco, solo, vittima, na, contento, zalone, parlavano, male, tolotolo, guarda, successo, non_fare, non_vittima, 💩]","[brodo, giugiule, tutto, farina, sacco, solo, vittima, na, contento, zalone, parlare, male, tolotolo, guardare, successo, non_fare, non_vittima, 💩]"
3,"[rimasti, male, cari, sovranisti, razzisti, 😅, stivali, rosicata, sovranisti, trollati, tolotolo, checcozalone, facciamorete, ❤, ️, 👊]","[rimanere, male, caro, sovranisti, razzista, 😅, stivale, rosicare, sovranisti, trollati, tolotolo, checcozalone, facciamorete, ❤, ️, 👊]"
4,"[salvini, propone, checcozalone, senatore, vita, dopo, aver, visto, trailer, tolotolo, film, rivela, fantasmagorica, presa, culo, salvinismo, zalone, sparisce, twet, bandierini, sovranisti, esilarante, oggi, comiche]","[salvini, proporre, checcozalone, senatore, vita, dopo, avere, visto, trailer, tolotolo, film, rivelare, fantasmagorico, presa, culo, salvinismo, zalone, sparire, bandierini, sovranisti, esilarante, oggi, comica]"


In [0]:
tweet27_8[['text','tokenLemma']]

,text,tokenLemma
0,"Comunque la si pensi, #Zalone riesce ancora una volta a riempire le sale di tutti i cinema italiani. In un settore da anni in grande difficoltà, lui riesce ad attirare spettatori di ogni età e classe sociale. È questa la grande forza di Checco: unire. #ToloTolo https://t.co/mUzLVlPxL0","[comunque, pensare, zalone, riuscire, ancora, volta, riempire, sale, cinema, italiano, settore, anno, grande, difficoltà, riuscire, attirare, spettatore, ogni, età, classe, sociale, grande, forza, checco, unire, tolotolo]"
1,"Non è ""diverso"" #ToloTolo. Sono i fan di #Zalone che finora non hanno capito un cazz*!","[non_diverso, non_tolotolo, fan, zalone, finora, non_capire, non_cazz]"
2,@matteosalvinimi @marcocf69 Sei in brodo di giugiule e tutta farina del tuo sacco fai solo la vittima na sei contento come #Zalone che parlavano male di #ToloTolo e guarda che successo .non fare la vittima!!💩,"[brodo, giugiule, tutto, farina, sacco, solo, vittima, na, contento, zalone, parlare, male, tolotolo, guardare, successo, non_fare, non_vittima, 💩]"
3,"Ci siete rimasti male ""cari"" Sovranisti=Razzisti\ndei miei stivali😅\n\nLa rosicata dei sovranisti trollati da #ToloTolo di #CheccoZalone\n\nhttps://t.co/6GKEMRVQJv di @neXtquotidiano \n\n#facciamorete ❤️🇮🇹👊 https://t.co/3UJtrKExAQ","[rimanere, male, caro, sovranisti, razzista, 😅, stivale, rosicare, sovranisti, trollati, tolotolo, checcozalone, facciamorete, ❤, ️, 👊]"
4,Salvini propone #CheccoZalone come senatore a vita dopo aver visto il trailer di #ToloTolo \nIl film si rivela una fantasmagorica presa per il culo del salvinismo \nZalone sparisce dai tweet di bandierini e sovranisti \nPiù esilarante di oggi le comiche,"[salvini, proporre, checcozalone, senatore, vita, dopo, avere, visto, trailer, tolotolo, film, rivelare, fantasmagorico, presa, culo, salvinismo, zalone, sparire, bandierini, sovranisti, esilarante, oggi, comica]"
...,...,...
21565,"Il prezzo del biglietto di #ToloTolo vale anche solo per il cameo di Nichi Vendola (che capiranno in pochissimi, come il vero senso del film)","[prezzo, biglietto, tolotolo, valere, solo, cameo, nichi, vendola, capire, poco, vero, senso, film]"
21566,Qualcuno si offende se affermo che #ToloTolo fa cagare?,"[qualcuno, offendere, affermare, tolotolo, fa, cagare]"
21567,"Siccome sul film di Zalone manca sempre l’opinione degli afrodiscendenti e quindi dei diretti interessati per ciò che concerne la rappresentazione dei corpi neri nei media e cinema italiani, ecco alcuni passaggi del post di Andi Nganso, medico e attivista:\n#ToloTolo https://t.co/SNYVUAx0dJ","[siccome, film, zalone, mancare, sempre, ’, opinione, afrodiscendenti, quindi, diretto, interessato, ciò, concernere, rappresentazione, corpo, nero, media, cinema, italiano, ecco, alcuni, passaggio, post, andi, nganso, medico, attivista]"
21568,"Matteo Salvini, non capendo proprio benissimo il significato della canzone ""Immigrato"", era arrivato a proporre #CheccoZalone come senatore a vita. \n\nOggi, il buon Checco - ed è una delle cose più tristi per chi fa satira - è dovuto arrivare a spiegarglielo: https://t.co/JFjNs854pM","[matteosalvini, non_capire, non_proprio, non_benissimo, non_significato, non_canzone, non_immigrare, arrivare, checcozalone, senatore, vita, oggi, checcozalone, cosa, triste, fare, satira, dovere, arrivare, spiegare]"


In [0]:
tweet27_8.to_csv('DatasetPreprocessato_finale.csv')